In [16]:
using Statistics
using Plots

In [17]:
ppal_dir = pwd()

function init_lattice(L, N, l)

    FCWS = []

    grid = zeros((L, L))
    
    idx = 0

    while idx < N

        x_head = rand(1 : (L - 1))

        x_tail = x_head + l - 1

        row = rand(1 : L)

        positions = [(row, mod(j, 1 : L)) for j in x_head : x_tail]

        validator = true

        #Check if any of these positions have been taken
        for pos in positions

            if grid[positions[1][1], positions[1][2]] == 1

                validator = false
                
            end

        end

        #if any position has been taken don't do anything
        if validator == false
            #DO nothing

        #If all the positions are free then take it
        else
            
            idx += 1

            for j in x_head : x_tail

                grid[row, mod(j, 1 : L)] = 1
                
            end

            append!(FCWS, [positions])

        end
    end

    return FCWS, grid

end


function move_walkers(L, N, l, FCWS, grid)
    
    directions = [(0,1), (0,-1), (1,0), (-1,0)]

    N_mov = 0

    k = 0
    
    @fastmath @inbounds for fcw in FCWS
        
        new_pos = [(0, 0) for j in 1 : l]

        k += 1
        
        dice = rand()

        if dice < 0.5
            
            #Choose 1 nearest neighbours poisition
            idx = rand(1 : 4)
            direction = directions[idx]

            #Move the tail (new head) to the chosen position
            move_to = (0, 0)

            move_to = (convert(Int32, mod(fcw[end][1] + direction[1], 1 : L)), convert(Int32, mod(fcw[end][2] + direction[2], 1 : L))) 

            #Only move if not occupied
            if grid[move_to[1], move_to[2]] != 1

                N_mov += 1

                #Update lattice for the head (new tail)
                grid[fcw[1][1], fcw[1][2]] = 0

                #Move the other particles following the head
                for i in 1 : l-1

                    new_pos[i] = fcw[i+1]

                end

                #Move the tail (new head)
                new_pos[end] = move_to

                #Update FCW position
                FCWS[k] = new_pos

                #Update lattice for tail (new head)
                grid[move_to[1], move_to[2]] = 1
                
            end
            
        else
        
            #Choose 1 nearest neighbours poisition
            idx = rand(1 : 4)
            direction = directions[idx]

            #Move the head to the chosen position
            move_to = (0, 0)

            move_to = (convert(Int32, mod(fcw[1][1] + direction[1], 1 : L)), convert(Int32, mod(fcw[1][2] + direction[2], 1 : L))) 

            #Only move if not occupied
            if grid[move_to[1], move_to[2]] != 1

                N_mov += 1

                #Update lattice for the tail
                grid[fcw[end][1], fcw[end][2]] = 0

                #Move the other particles following the head
                for i in 2 : l

                    new_pos[i] = fcw[i-1]

                end

                #Move the head
                new_pos[1] = move_to

                #Update FCW position
                FCWS[k] = new_pos

                #Update lattice for head
                grid[move_to[1], move_to[2]] = 1
                
            end
                    
        end
    end
    
    mob = N_mov / N
    
    return mob, grid, FCWS
        
end

function simulate(L, N, l, t)

    FCWS, grid = init_lattice(L, N, l)
    
    heatmap(grid)

    mobility_arr = zeros(t)

    for k in 1 : t

        percentage = round(k/t * 100, digits=2)
        
        #println(percentage, " %", " done")

        mobility, grid, FCWS = move_walkers(L, N, l, FCWS, grid)
        
        @inbounds mobility_arr[k] = mobility
            
    end

    return mobility_arr, grid
        
end

simulate (generic function with 1 method)

# Single density simulation

In [18]:
function single_density(L, N, l, t, folder="Data")

    t0 = time_ns() #Start simulation

    mobility, grid = simulate(L, N, l, t)

    tf = time_ns() #Simulation finished

    ET = round((tf - t0) * 1e-9, sigdigits=3) #Compute elapsed time
    
    #Create folder if doesn't exist
    if ! isdir(folder)
        mkdir(folder)
    end
    
    #Choose folder
    cd(folder)
    
    #Write parameters used to file
    f_parameters = open("parameters_used.txt", "w")
    
    println(f_parameters, "L: ", L)
    println(f_parameters, "l: ", l)
    println(f_parameters, "N: ", N)
    println(f_parameters, "t: ", t)
    
    println(f_parameters, "\nElapsed time: ", ET)
    
    close(f_parameters)
    
    cd(ppal_dir) #Return to home directory
    
    println("Elapsed time: ", ET, " s")
    
    return mobility, grid

end

single_density (generic function with 2 methods)

# Several densities simulations

In [19]:
function several_densities(L, l, t, eq_t, times, densities, folder="Data")

    t0 = time_ns()

    k = 0
    
    if ! isdir(folder)
        mkdir(folder)
    end
    
    cd(folder)
    
    f = open("mobility.txt", "w")
    
    println(f, "#<M(t)>\tρ")
    println("#mobility\tdensity")
    
    #Simulate and compute mobility for each density
    for density in densities

        k += 1

        N = convert(Int, round(L^2 * density / l, digits=0))
        
        mean_mob = 0.
        
        #Average over "times" realisations
        for i in 1 : times
    
            mobility, grid = simulate(L, N, l, t)

            mean_mob += mean(mobility[eq_t : end])
            
        end
        
        mean_mob = mean_mob / times
        
        println(f, mean_mob, "\t", density)
        println(mean_mob, "\t", density)

    end
    
    close(f)
    
    tf = time_ns()
    
    ET = round((tf - t0) * 1e-9, sigdigits=3)
    
    f_parameters = open("parameters_used.txt", "w")
    
    println(f_parameters, "L: ", L)
    println(f_parameters, "l: ", l)
    println(f_parameters, "t: ", t)
    println(f_parameters, "Eq_t: ", eq_t)
    println(f_parameters, "times: ", times)
    
    println(f_parameters, "\nElapsed time: ", ET)
    
    close(f_parameters)
    
    cd(ppal_dir)

    println("Elapsed time: ", ET, " s")
    
end

several_densities (generic function with 2 methods)

# Run simulation 

In [20]:
L = 100
l = 1

t = 10^4
eq_t = 10^3

times = 1

densities = collect(0.01: 0.01 : 0.8)

foldername = "l_$l"

several_densities(L, l, t, eq_t, times, densities, foldername)

#mobility	density
0.9903544050661038	0.01
0.9803177424730584	0.02
0.9700462911528349	0.03
0.9601430396622599	0.04
0.9500346628152428	0.05
0.9401098026145243	0.06
0.9300496770200135	0.07
0.9202852460837684	0.08
0.9102658963818835	0.09
0.9000471058771249	0.1
0.8902217935380917	0.11
0.8802594156204866	0.12
0.870023758043978	0.13
0.8600788007681684	0.14
0.8500045920823612	0.15
0.839923480724364	0.16
0.8303596985955809	0.17
0.8200450567220926	0.18
0.8102333074102878	0.19
0.8000032774136205	0.2
0.7900988249982807	0.21
0.7805714011574472	0.22
0.7699856054641271	0.23
0.7596201810909898	0.24
0.750361559826686	0.25
0.7399193679334775	0.26
0.7300241125471655	0.27
0.7199564730902915	0.28
0.7100702603925234	0.29
0.7002860052586749	0.3
0.6901103461622543	0.31
0.6801023844572825	0.32
0.6702022670881687	0.33
0.6602952939869428	0.34
0.6501224625835226	0.35
0.6398967398684098	0.36
0.630316241138372	0.37
0.6199476958700495	0.38
0.6102142496987514	0.39
0.6002646372625264	0.4
0.5900015987383516	0.41
0.5798